# get_fdt test

## TO-DO
- sort by values or by index
- fmt_values

In [1]:
## Standard Libs
from typing import Union, Optional, Tuple, Literal, Any
import warnings
import random

# Third-Party Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

# # Local Libs
# from jm_datascience import jm_pandas as jm_pd
# from jm_datascience import jm_pdaccessor as jm
# from jm_utils import jm_richprt as jm_prt


## Some Series and DFs

In [2]:
df_work = pd.DataFrame({
    'nombre': ['Ana', 'Bob', '', 'Carlos', ' ', 'Diana'],
    'apellido': ['A_Ana', 'B_Bob', None, 'C_Carlos', None, 'D_Diana'],
    'edad': [25, -1, 30, 999, 28, 22],
    'ciudad': ['Madrid', 'N/A', 'Barcelona', 'Valencia', 'unknown', 'Sevilla'],
    'salario': [50000, 0, 60000, -999, 55000, 48000]
})

## Read spreedsheet for tests
try:
    spreedsheet = r"C:\Users\jm\Documents\__Dev\PortableGit\__localrepos\365DS_jm\3_statistics\2_13_Practical_Ex_Descriptive_Stats.xlsx"    # Casa
    with open(spreedsheet) as f:
        pass
except FileNotFoundError:
    spreedsheet = r"D:\git\PortableGit\__localrepos\365DS_jm\3_statistics\2_13_Practical_Ex_Descriptive_Stats.xlsx"                         # Office

df_xls = pd.read_excel(spreedsheet, skiprows=4, usecols='B:J,L:AA', index_col='ID')
df = df_xls.copy()

lst_str = random.choices([chr(i) for i in range(65, 72)], k=175)
# sr_str = jm_pd.to_series(lst_str)                         # <- jm_pd.to_serie_with_count()

In [3]:
display(df['State'].unique())
df['State'].value_counts(sort=False, ascending=True)
df['Country'].value_counts(sort=True, ascending=True, dropna=False)

array(['California', 'Virginia', 'Arizona', 'Oregon', 'Nevada',
       'Colorado', 'Utah', nan, 'Kansas', 'Wyoming'], dtype=object)

Country
Germany      1
Mexico       1
Denmark      1
UK           2
Belgium      2
Russia       4
Canada       7
USA         12
NaN         72
USA        165
Name: count, dtype: int64

In [4]:
def _fmt_value_for_pd(value, width=8, decimals=3, miles=',') -> str:
    """
    Format a value (numeric or string) into a right-aligned string of fixed width.

    Converts numeric values to formatted strings with thousands separators and
    specified decimal places. Strings are padded to the same width for consistent alignment.

    Parameters:
        value (int, float, str): The value to be formatted.
        width (int): Total width of the output string. Must be a positive integer.
        decimals (int): Number of decimal places for numeric values. Must be >= 0.
        miles (str or None): Thousands separator. Valid options: ',', '_', or None.

    Returns:
        str: The formatted string with right alignment.

    Raises:
        ValueError: If width <= 0, decimals < 0, or miles is invalid.

    Examples:
        >>> format_value(123456.789)
        '123,456.79'
        >>> format_value("text", width=10)
        '      text'
        >>> format_value(9876, miles=None)
        '    9876.00'
    """
    # Parameter Value validation <- vamos a tener que analizar este tema por si es un list , etc,,
    #   - En realidad acá tenemos que evaluar algo similar a jm_utils - fmt_values() FUTURE
    # if not isinstance(value, (int, float, np.integer, np.floating)) or pd.api.types.is_any_real_numeric_dtype(value)

    if not isinstance(width, int) or width <= 0:
        raise ValueError(f"Width must be a positive integer. Not '{width}'")
    
    if not isinstance(decimals, int) or decimals < 0:
        raise ValueError(f"Decimals must be a non-negative integer. Not '{decimals}")
    
    if miles not in [',', '_', None]:
        raise ValueError(f"Miles must be either ',', '_', or None. Not '{miles}")
    
    try:
        num = float(value)                                  # Convert to float if possible
        if num % 1 == 0:                                    # it its a total integer number
            decimals = 0
        if miles:
            return f"{num:>{width}{miles}.{decimals}f}"     # Ancho fijo, x decimales, alineado a la derecha
        else:
            return f"{num:>{width}.{decimals}f}"
        
    except (ValueError, TypeError):
        return str(value).rjust(width)                      # Alinea también strings, para mantener la grilla

In [5]:
def to_series(
    data: Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame],
    index: Optional[pd.Index] = None,
    name: Optional[str] = None
) -> pd.Series:
    """
    Converts input data into a pandas Series, optionally returning value counts.

    This function accepts various data types and converts them into a pandas Series.
    If `count=True`, it returns the frequency count of the values in the resulting Series.

    Parameters:
        data (Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame]):
            The input data to convert. Supported types include:
            - pd.Series: returned as-is or counted if `count=True`.
            - np.ndarray: flattened and converted to a Series.
            - dict: keys become the index, values are used for data.
            - list or set: converted directly to a Series.
            - pd.DataFrame:
                - 1 column: converted directly to a Series.
                - 2 columns: first column becomes the index, second becomes the values.

        count (bool or int, optional): Whether to return value counts instead of raw data.
            If True or 1, returns frequencies of each value. Default is False.

    Returns:
        pd.Series: A pandas Series representing the input data. If `count=True`, returns
            the value counts of the data.

    Raises:
        TypeError: If `data` is not one of the supported types.
        ValueError: If `count` is not a boolean or integer 0/1.
        ValueError: If DataFrame has more than 2 columns.

    Examples:
        >>> import pandas as pd
        >>> to_serie_with_count([1, 2, 2, 3])
        0    1
        1    2
        2    2
        3    3
        dtype: int64

        >>> to_serie_with_count([1, 2, 2, 3], count=True)
        2    2
        1    1
        3    1
        dtype: int64

        >>> df = pd.DataFrame({'Category': ['A', 'B', 'A'], 'Value': [10, 20, 30]})
        >>> to_serie_with_count(df)
        Category
        A    10
        B    20
        A    30
        Name: Value, dtype: int64
    """
    
    # Validate parameters - FUTURE
    
    if isinstance(data, pd.Series):                 # If data is already a series no conversion needed
        sr = data                                  
    elif isinstance(data, np.ndarray):              # If data is a NumPy array   
        sr = pd.sr(data.flatten())
    elif isinstance(data, (dict, list)):
        sr = pd.sr(data)
    elif isinstance(data, (set)):
        sr = pd.sr(tuple(data))
    elif isinstance(data, pd.DataFrame):
        if data.shape[1] == 1:                      # Also len(data.columns == 1)
            sr = data.iloc[:, 0]
        elif data.shape[1] == 2:                    # Index: first col, Data: 2nd Col
            sr = data.set_index(data.columns[0])[data.columns[1]]
        else:
            raise ValueError("DataFrame must have 1 oer 2 columns. Categories and values for 2 columns cases.")
    else:
        raise TypeError(f"Unsupported data type: {type(data)}. "
                    "Supported types: pd.sr, np.ndarray, pd.DataFrame, dict, list, set, and pd.DataFrame")

    if name:
        sr.name = name

    if index:
        sr.index = index

    return sr

In [6]:
def _validate_numeric_series(
        data: Union[pd.Series, pd.DataFrame],
        positive: Optional[bool] = True
) -> Union[None, Exception]:

    # Validate data parameter a pandas object
    if not isinstance(data, (pd.Series, pd.DataFrame)):     # pd.Series or pd.Datafram
        raise TypeError(
            f"Input data must be a pandas Series or DataFrame. Got {type(data)} instead."
        )
              
    if positive:
        if not all(                                             # Only positve numeric values                 
            isinstance(val, (int, float, np.integer, np.floating)) and val > 0 for val in data.values
        ):
            raise ValueError(f"All values in 'data' must be positive numeric values.")
        pass
    else:                                                       # Just only numeric values
        if not all(isinstance(val, (int, float, np.integer, np.floating)) for val in data.values):
            raise ValueError(f"All values in 'data' must be numeric values.")
        pass

## OJO - podemos refactor get_fdt
- Podemos impactar el 'order' directamente a la fdt resultante y no a la serie previa
- De esta manera podemo hacer que el valor del aside también se ordene pero NOOO.. mejor que si es aside quede al final!

In [7]:
def get_fdt0(
        data: Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame],
        value_counts: Optional[bool] = False,
        dropna: Optional[bool] = True,
        na_position: Optional[str] = 'last',
        include_pcts: Optional[bool] = True,
        include_plain_relatives: Optional[bool] = True,
        fmt_values: Optional[bool] = False,
        order: Optional[str] = 'desc',
        na_aside: Optional[bool] = True
) -> pd.DataFrame:
    """
    Generates a Frequency Distribution Table (FDT) with absolute, relative, and cumulative frequencies.

    This function converts various input data types into a structured DataFrame containing:
    - Absolute frequencies
    - Cumulative frequencies
    - Relative frequencies (proportions and percentages)
    - Cumulative relative frequencies (percentages)

    Parameters:
        data (Union[pd.Series, np.ndarray, dict, list, pd.DataFrame]): Input data.
            If DataFrame, it will be converted to a Series using `to_series`.
        value_counts (bool, optional): Whether to count occurrences if input is raw data.
            Assumes data is not pre-counted. Default is False.
        dropna (bool, optional): Whether to exclude NaN values when counting frequencies.
            Default is True.
        na_position (str, optional): Position of NaN values in the output:
            - 'first': Place NaN at the top.
            - 'last': Place NaN at the bottom (default).
            - 'value': Keep NaN in its natural order.
            Default is 'last'.
        include_pcts (bool, optional): Whether to include percentage columns.
            If False, only absolute and cumulative frequencies are returned.
            Default is True.
        include_plain_relatives (bool, optional): Whether to return relative and cumulative relative values.
            If False, only frequency and percentage columns are included.
            Default is True.
        fmt_values (bool, optional): Whether to format numeric values using `_fmt_value_for_pd`.
            Useful for improving readability in reports. Default is False.
        order (str, optional): Sort order for the output:
            - 'asc': Sort values ascending.
            - 'desc': Sort values descending (default).
            - 'ix_asc': Sort by index ascending.
            - 'ix_desc': Sort by index descending.
            - None: No sorting.
            Default is 'desc'.
        na_aside (bool, optional): Whether to separate NaN values from calculations but keep them in the output.
            If True, NaNs are added at the end and not included in cumulative or relative calculations.
            Default is True.

    Returns:
        pd.DataFrame: A DataFrame containing the frequency distribution table with the following columns
        (depending on parameters):
            - Frequency
            - Cumulative Frequency
            - Relative Frequency
            - Cumulative Relative Freq.
            - Relative Freq. [%]
            - Cumulative Freq. [%]

    Raises:
        ValueError: If `sort` or `na_position` receive invalid values.

    Notes:
        - This function uses `to_series` to convert input data into a pandas Series.
        - If `na_aside=True` and NaNs are present, they are placed separately and not included in relative calculations.
        - Useful for exploratory data analysis and generating clean statistical summaries.

    Example:
        >>> import pandas as pd
        >>> data = pd.Series(['A', 'B', 'A', 'C', 'B', 'B', None])
        >>> fdt = get_fdt(data, sort='desc', fmt_values=True)
        >>> print(fdt)
              Frequency  Cumulative Frequency  Relative Freq. [%]  Cumulative Freq. [%]
        B           3                   3                42.86                  42.86
        A           2                   5                28.57                  71.43
        C           1                   6                14.29                  85.71
        Nulls       1                   7                14.29                 100.00
    """
    columns = [
        'Frequency',
        'Cumulative Frequency',
        'Relative Frequency',
        'Cumulative Relative Freq.',
        'Relative Freq. [%]',
        'Cumulative Freq. [%]'
    ]
    # def _calculate_fdt_relatives(series):     # Revisar, no me gusta el flujo actual
    
    sr = to_series(data)
    
    if dropna:
        sr = sr.dropna()                        # Drop all nulls values of the Series
        sr = sr.drop(np.nan, errors='ignore')   # For series with NaNs as a category with their count (errors='ignore': does not fail if it does not exist)

    if value_counts:
        sr = sr.value_counts(dropna=dropna, sort=False)

    # Validate that all the values are positive numbers
    if not _validate_numeric_series(sr):
        raise ValueError(f"To get a Frequency Distribution Table all frequencies must by positive numbers")

    # Order de original Series to obtain the fdt in the same order as the original data
    match order:
        case 'asc':
            sr = sr.sort_values()
        case 'desc':
            sr = sr.sort_values(ascending=False)
        case 'ix_asc':
            sr = sr.sort_index()
        case 'ix_desc':
            sr = sr.sort_index(ascending=False)
        case None:
            pass
        case _:
            raise ValueError(f"Valid values for order: 'asc', 'desc', 'ix_asc', 'ix_desc', or None. Got '{order}'")

    # Handle NaN values 
    try:                            # To manage when there aren't NaNs
        nan_value = sr[np.nan]
        sr_without_nan = sr.drop(np.nan)
    except:
        nan_value = 0
        sr_without_nan = sr.copy()  # If no NaNs, we keep the original series without changes
    finally:
        if na_aside:
            # Column that will then be concatenated to the end of the DF if the na_aside option is true
            nan_row_df = pd.DataFrame(data = [nan_value], columns=[columns[0]], index=['Nulls'])      # Only 'Frequency' column.
            if nan_value > 0:
                sr = sr_without_nan

    match na_position and not na_aside:              # Locate the NaNs values
        case 'first':
            sr = pd.concat([pd.Series({np.nan: nan_value}), sr_without_nan])
        case 'last':
            sr = pd.concat([sr_without_nan, pd.Series({np.nan: nan_value})])
        case 'value' | None:
            pass
        case _:
            raise ValueError(f"Valid values for na_position: 'first', 'last', 'value' or None. Got '{na_position}'")

    # Central rutine: create the fdt, including relative and cumulative columns.
    fdt = pd.DataFrame(sr)
    fdt.columns = [columns[0]]
    fdt[columns[1]] = fdt['Frequency'].cumsum()
    fdt[columns[2]] = fdt['Frequency'] / fdt['Frequency'].sum()
    fdt[columns[3]] = fdt['Relative Frequency'].cumsum()
    fdt[columns[4]] = fdt['Relative Frequency'] * 100
    fdt[columns[5]] = fdt['Cumulative Relative Freq.'] * 100

    if na_aside and not dropna:             # We add nan_columns at the end
        fdt = pd.concat([fdt, nan_row_df])

    if not include_pcts:                    # Don't return percentage columns
        fdt = fdt[columns[0:4]]
    
    if not include_plain_relatives:         # Don't return relative and plain cumulative
        fdt = fdt[[columns[0], columns[4], columns[5]]]

    if fmt_values:
        fdt = fdt.map(_fmt_value_for_pd)
        
    return fdt

## OJO - podemos refactor get_fdt
- Podemos impactar el 'order' directamente a la fdt resultante y no a la serie previa
- De esta manera podemo hacer que el valor del aside también se ordene pero NOOO.. mejor que si es aside quede al final!

In [14]:
def get_fdt(
        data: Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame],
        value_counts: Optional[bool] = False,
        dropna: Optional[bool] = True,
        na_position: Optional[str] = 'last',
        include_pcts: Optional[bool] = True,
        include_plain_relatives: Optional[bool] = True,
        fmt_values: Optional[bool] = False,
        order: Optional[str] = 'desc',
        na_aside: Optional[bool] = True
) -> pd.DataFrame:
    """
    Generates a Frequency Distribution Table (FDT) with absolute, relative, and cumulative frequencies.

    This function converts various input data types into a structured DataFrame containing:
    - Absolute frequencies
    - Cumulative frequencies
    - Relative frequencies (proportions and percentages)
    - Cumulative relative frequencies (percentages)

    Parameters:
        data (Union[pd.Series, np.ndarray, dict, list, pd.DataFrame]): Input data.
            If DataFrame, it will be converted to a Series using `to_series`.
        value_counts (bool, optional): Whether to count occurrences if input is raw data.
            Assumes data is not pre-counted. Default is False.
        dropna (bool, optional): Whether to exclude NaN values when counting frequencies.
            Default is True.
        na_position (str, optional): Position of NaN values in the output:
            - 'first': Place NaN at the top.
            - 'last': Place NaN at the bottom (default).
            - 'value': Keep NaN in its natural order.
            Default is 'last'.
        include_pcts (bool, optional): Whether to include percentage columns.
            If False, only absolute and cumulative frequencies are returned.
            Default is True.
        include_plain_relatives (bool, optional): Whether to return relative and cumulative relative values.
            If False, only frequency and percentage columns are included.
            Default is True.
        fmt_values (bool, optional): Whether to format numeric values using `_fmt_value_for_pd`.
            Useful for improving readability in reports. Default is False.
        order (str, optional): Sort order for the output:
            - 'asc': Sort values ascending.
            - 'desc': Sort values descending (default).
            - 'ix_asc': Sort by index ascending.
            - 'ix_desc': Sort by index descending.
            - None: No sorting.
            Default is 'desc'.
        na_aside (bool, optional): Whether to separate NaN values from calculations but keep them in the output.
            If True, NaNs are added at the end and not included in cumulative or relative calculations.
            Default is True.

    Returns:
        pd.DataFrame: A DataFrame containing the frequency distribution table with the following columns
        (depending on parameters):
            - Frequency
            - Cumulative Frequency
            - Relative Frequency
            - Cumulative Relative Freq.
            - Relative Freq. [%]
            - Cumulative Freq. [%]

    Raises:
        ValueError: If `sort` or `na_position` receive invalid values.

    Notes:
        - This function uses `to_series` to convert input data into a pandas Series.
        - If `na_aside=True` and NaNs are present, they are placed separately and not included in relative calculations.
        - Useful for exploratory data analysis and generating clean statistical summaries.

    Example:
        >>> import pandas as pd
        >>> data = pd.Series(['A', 'B', 'A', 'C', 'B', 'B', None])
        >>> fdt = get_fdt(data, sort='desc', fmt_values=True)
        >>> print(fdt)
              Frequency  Cumulative Frequency  Relative Freq. [%]  Cumulative Freq. [%]
        B           3                   3                42.86                  42.86
        A           2                   5                28.57                  71.43
        C           1                   6                14.29                  85.71
        Nulls       1                   7                14.29                 100.00
    """
    columns = [
        'Frequency',
        'Cumulative Frequency',
        'Relative Frequency',
        'Cumulative Relative Freq.',
        'Relative Freq. [%]',
        'Cumulative Freq. [%]'
    ]
    # def _calculate_fdt_relatives(series):     # Revisar, no me gusta el flujo actual
    
    sr = to_series(data)
    
    if dropna:
        sr = sr.dropna()                        # Drop all nulls values of the Series
        sr = sr.drop(np.nan, errors='ignore')   # For series with NaNs as a category with their count (errors='ignore': does not fail if it does not exist)

    if value_counts:
        sr = sr.value_counts(dropna=dropna, sort=False)

    # Validate that all the values are positive numbers
    _validate_numeric_series(sr)

    # Order de original Series to obtain the fdt in the same order as the original data
    match order:
        case 'asc':
            sr = sr.sort_values()
        case 'desc':
            sr = sr.sort_values(ascending=False)
        case 'ix_asc':
            sr = sr.sort_index()
        case 'ix_desc':
            sr = sr.sort_index(ascending=False)
        case None:
            pass
        case _:
            raise ValueError(f"Valid values for order: 'asc', 'desc', 'ix_asc', 'ix_desc', or None. Got '{order}'")
        
    # Handle NaNs values. Two cases: 1. na_aside: don't use for calcs and at the end; 2. use for calcs and locate according na_position
    #   - Determine the number of nans
    if pd.isna(sr.index).any():
        n_nans = sr[np.nan]
    else:
        n_nans = 0

    #   - Locale NaNs row in the Series 'sr'
    if na_aside:
        sr = sr.drop(np.nan, errors='ignore')
        # Column that will then be concatenated to the end of the DF - Only 'Frequency' column, no calculated columns
        nan_row_df = pd.DataFrame(data = [n_nans], columns=[columns[0]], index=[np.nan])
        # nan_row_df = pd.DataFrame(data = [n_nans], columns=[columns[0]], index=['Nulls'])
    else:
        # As we use NaNs for calculations decide where locate these values
        sr_without_nan = sr.drop(np.nan, errors='ignore')
        match na_position:             
            case 'first':
                sr = pd.concat([pd.Series({np.nan: n_nans}), sr_without_nan])
            case 'last':
                sr = pd.concat([sr_without_nan, pd.Series({np.nan: n_nans})])
            case 'value' | None:
                pass                # Locates the Nulls row based on the value or index ordering
            case _:
                raise ValueError(f"Valid values for na_position: 'first', 'last', 'value' or None. Got '{na_position}'")

    # Central rutine: create the fdt, including relative and cumulative columns.
    fdt = pd.DataFrame(sr)
    fdt.columns = [columns[0]]
    fdt[columns[1]] = fdt['Frequency'].cumsum()
    fdt[columns[2]] = fdt['Frequency'] / fdt['Frequency'].sum()
    fdt[columns[3]] = fdt['Relative Frequency'].cumsum()
    fdt[columns[4]] = fdt['Relative Frequency'] * 100
    fdt[columns[5]] = fdt['Cumulative Relative Freq.'] * 100

    if na_aside and not dropna:             # We add nan_columns at the end
        fdt = pd.concat([fdt, nan_row_df])

    if not include_pcts:                    # Don't return percentage columns
        fdt = fdt[columns[0:4]]
    
    if not include_plain_relatives:         # Don't return relative and plain cumulative
        fdt = fdt[[columns[0], columns[4], columns[5]]]

    if fmt_values:
        fdt = fdt.map(_fmt_value_for_pd)
        
    return fdt

In [15]:
# fdt_s1 = get_fdt(df['State'], value_counts=True, dropna=False, na_position='last', include_pcts=True, include_plain_relatives=True, fmt_values=True, order='desc', na_aside=True)
# fdt_s1 = get_fdt(df['Country'], value_counts=True, dropna=False, na_position='last', include_pcts=True, include_plain_relatives=True, fmt_values=True, order='desc', na_aside=True)
fdt_g1 = get_fdt(df['Gender'], value_counts=True, dropna=False, na_position='value', include_pcts=True, include_plain_relatives=True, fmt_values=True, order='desc', na_aside=False)
fdt_g1


,Frequency,Cumulative Frequency,Relative Frequency,Cumulative Relative Freq.,Relative Freq. [%],Cumulative Freq. [%]
Gender,,,,,,
M,108,108,0.404,0.404,40.449,40.449
NaN,89,197,0.333,0.738,33.333,73.783
F,70,267,0.262,1,26.217,100


In [16]:
fdt_s1 = get_fdt(df['State'], value_counts=True, dropna=False, na_position='value', include_pcts=True, include_plain_relatives=True, fmt_values=True, order='desc', na_aside=True)
fdt_s1


,Frequency,Cumulative Frequency,Relative Frequency,Cumulative Relative Freq.,Relative Freq. [%],Cumulative Freq. [%]
California,119,119,0.657,0.657,65.746,65.746
Nevada,17,136,0.094,0.751,9.392,75.138
Colorado,11,147,0.061,0.812,6.077,81.215
Oregon,11,158,0.061,0.873,6.077,87.293
Arizona,11,169,0.061,0.934,6.077,93.370
Utah,6,175,0.033,0.967,3.315,96.685
Virginia,4,179,0.022,0.989,2.210,98.895
Kansas,1,180,0.006,0.994,0.552,99.448
Wyoming,1,181,0.006,1.000,0.552,100.000
NaN,86,nan,nan,nan,nan,nan


In [19]:
vc = df['State'].value_counts(sort=False, dropna=True)
fdt_vc = get_fdt(vc, dropna=True, na_position='value', include_pcts=True, include_plain_relatives=True, fmt_values=True, order='desc', na_aside=True)
fdt_vc

,Frequency,Cumulative Frequency,Relative Frequency,Cumulative Relative Freq.,Relative Freq. [%],Cumulative Freq. [%]
State,,,,,,
California,119,119,0.657,0.657,65.746,65.746
Nevada,17,136,0.094,0.751,9.392,75.138
Arizona,11,147,0.061,0.812,6.077,81.215
Oregon,11,158,0.061,0.873,6.077,87.293
Colorado,11,169,0.061,0.934,6.077,93.370
Utah,6,175,0.033,0.967,3.315,96.685
Virginia,4,179,0.022,0.989,2.210,98.895
Kansas,1,180,0.006,0.994,0.552,99.448
Wyoming,1,181,0.006,1.000,0.552,100.000


In [12]:
# # vc.dropna()
# vc.drop(np.nan, errors='ignore')
# # nans = vc[np.nan]

In [13]:
fdt_vc1 = get_fdt(vc, value_counts=True, dropna=False, na_position='last', include_pcts=True, include_plain_relatives=True, fmt_values=True, order='desc', na_aside=True)
# fdt_vc1 = get_fdt(vc, na_aside=False)
fdt_vc1

NameError: name 'vc' is not defined

In [ ]:
# fdt_s1 = get_fdt(df['Country'], value_counts=True, sort='asc', dropna=False, na_position='value', fmt_values=True, na_aside=False)
# fdt_s1
fdt_s2 = get_fdt(df['State'], value_counts=True, dropna=False, na_aside=False, na_position='value', fmt_values=True)
fdt_s2



In [ ]:
fdt_2 = get_fdt(df['State'], value_counts=True)    
cumulative_pcts = fdt_2['Cumulative Freq. [%]']
top_3_pct = cumulative_pcts.iloc[min(2, len(cumulative_pcts)-1)]

labels = [f"{fdt_2.iloc[ix, 0]} ({fdt_2.iloc[ix, -2]:.1f} %)" for ix in range(fdt_2.shape[0])]
print(labels)

for iloc_ix in range(len(cumulative_pcts)):
    print(f"cumulative_pcts.iloc[{iloc_ix}] = {cumulative_pcts.iloc[iloc_ix]}")

display(len(cumulative_pcts))
display(top_3_pct)
fdt_2

## Some Typing Tests

In [ ]:
from typing import Union, Optional, Any, Literal, Sequence, TypeAlias
import pandas as pd

IndexElement: TypeAlias = Union[str, int, float, 'datetime.datetime']

def to_series(
    data: Union[pd.Series, np.ndarray, dict, list, set, pd.DataFrame],
    index: Optional[Union[pd.Index, Sequence[IndexElement]]] = None,
    name: Optional[str] = None
) -> pd.Series:
    """
    Converts input data into a pandas Series, optionally returning value counts.
    """
    return pd.Series(data, index=index, name=name)


to_series([1, 2, 3], ['a', 'b', 'c'], name='example_series')

In [ ]:
## Some Typing Tests
## Standard Libs
from typing import Union, Optional, Any, Literal, Sequence, TypeAlias

# Third-Party Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter  # for pareto chart and ?
import seaborn as sns
## Claude - Qwen


## Custom types for non-included typing annotations - Grok
IndexElement: TypeAlias = Union[str, int, float, 'datetime.datetime', pd.Timestamp]


def test_typing(
        value: Union[int, float, str],
        data: Optional[Union[pd.Index, Sequence[IndexElement]]] = None,
    ) -> None:
    """
    Test function to demonstrate typing with Union.

    Parameters:
        value (Union[int, float, str]): The input value which can be an int, float, or str.

    Returns:
        str: A string representation of the input value.
    """
    if isinstance(value, (int, float)):
        print(f"Numeric value: {value}")
    elif isinstance(value, str):
        print(f"String value: {value}")
    else:
        raise TypeError(f"Unsupported type: {type(value)}")
    
    if data is not None:
        if isinstance(data, pd.Index):
            print(f"Data is a pandas Index with {len(data)} elements.")
        elif isinstance(data, (list, tuple, np.ndarray)):
            print(f"Data is a sequence with {len(data)} elements.")
        else:
            raise TypeError(f"Unsupported data type: {type(data)}")
    

In [ ]:
test_typing(42, data=pd.Index(['a', 'b', 'c']))


In [ ]:
from typing import TypeAlias, Optional, Union, Sequence
import pandas as pd
import numpy as np
from numpy.typing import NDArray

IndexElement: TypeAlias = Union[str, int, float, 'datetime.datetime', np.str_, np.int64, np.float64, np.datetime64]
IndexLike: TypeAlias = Union[pd.Index, Sequence[IndexElement], NDArray[IndexElement]]

def mi_funcion(data, index: Optional[IndexLike] = None) -> None:
    if index is not None:
        if isinstance(index, np.ndarray) and index.ndim != 1:
            raise ValueError("El array de NumPy debe ser 1D para usarse como índice")
        index = pd.Index(index) if not isinstance(index, pd.Index) else index
        print("Índice proporcionado:", index)
    else:
        print("No se proporcionó índice, usando índice por defecto")

In [ ]:
mi_funcion(data=pd.Series([1, 2, 3]), index=np.array(['a', 'b', 'c']))
mi_funcion(data=pd.Series([1, 2, 3]), index=pd.Index(['a', 'b', 'c']))
mi_funcion(data=pd.Series([1, 2, 3]))  # Sin índice